In [1]:

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
df = pd.read_csv("final_dataset.csv")

# Konversi tanggal menjadi datetime
df[['pangan', 'provinsi', 'tanggal']] = df['id'].str.split('/', expand=True)
df['tanggal'] = pd.to_datetime(df['tanggal'], format='%Y-%m-%d')

# Tambahkan fitur tanggal: year, month, day
df['year'] = df['tanggal'].dt.year
df['month'] = df['tanggal'].dt.month
df['day'] = df['tanggal'].dt.day

# Hapus kolom yang tidak diperlukan setelah ekstraksi tanggal
df.drop(columns=['pangan', 'provinsi', 'tanggal'], inplace=True)

# Tangani missing values pada 'price' dengan median
df.fillna({'price': df['price'].median()}, inplace=True)

# Pisahkan fitur (X) dan target (y)
X, y = df.drop(columns=['id']), df['price']

# Tampilkan hasil preprocessing
df.head()

id    price  year  month  day
0  Bawang Merah/Aceh/2022-01-01  28970.0  2022      1    1
1  Bawang Merah/Aceh/2022-01-02  29900.0  2022      1    2
2  Bawang Merah/Aceh/2022-01-03  28970.0  2022      1    3
3  Bawang Merah/Aceh/2022-01-04  29600.0  2022      1    4
4  Bawang Merah/Aceh/2022-01-05  29540.0  2022      1    5

In [3]:
# Bagi dataset menjadi training dan test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Konversi ke DMatrix untuk XGBoost
train_dmatrix = xgb.DMatrix(data=X_train, label=y_train)
test_dmatrix = xgb.DMatrix(data=X_test)

# Parameter XGBoost
params = {'objective': 'reg:squarederror',
          'max_depth': 5,
          'learning_rate': 0.05,
          'eval_metric': 'rmse'}

# Train model
num_round = 100
model = xgb.train(params, train_dmatrix, num_round)

# Prediksi pada data uji
y_pred_test = model.predict(test_dmatrix)

# Hitung RMSE dan MAE
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae = mean_absolute_error(y_test, y_pred_test)
mape = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100

# Cetak hasil evaluasi
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")

MAE: 214.28
RMSE: 523.78
MAPE: 0.65%


In [4]:
# Prediksi untuk seluruh dataset
full_dmatrix = xgb.DMatrix(data=X)
y_pred_full = model.predict(full_dmatrix)

# Bulatkan hasil prediksi
df['price'] = [round(value) for value in y_pred_full]

# Simpan hasil prediksi ke file CSV
df[['id', 'price']].to_csv("prediksi_harga.csv", index=False)

# Tampilkan beberapa hasil pertama
print(df[['id', 'price']].head())

                             id  price
0  Bawang Merah/Aceh/2022-01-01  29131
1  Bawang Merah/Aceh/2022-01-02  30000
2  Bawang Merah/Aceh/2022-01-03  29131
3  Bawang Merah/Aceh/2022-01-04  29561
4  Bawang Merah/Aceh/2022-01-05  29561


In [ ]:

# Load dataset sample submission
df_sample = pd.read_csv("sample_submission.csv")

# Lakukan preprocessing yang sama pada sample submission
df_sample[['pangan', 'provinsi', 'tanggal']] = df_sample['id'].str.split('/', expand=True)
df_sample['tanggal'] = pd.to_datetime(df_sample['tanggal'], format='%Y-%m-%d')

df_sample['year'] = df_sample['tanggal'].dt.year
df_sample['month'] = df_sample['tanggal'].dt.month
df_sample['day'] = df_sample['tanggal'].dt.day

df_sample.drop(columns=['pangan', 'provinsi', 'tanggal'], inplace=True)

# Pastikan dataset sample memiliki fitur yang sesuai dengan model
X_sample = df_sample.drop(columns=['id'], errors='ignore')

# Konversi ke DMatrix dan prediksi
sample_dmatrix = xgb.DMatrix(data=X_sample)
df_sample['price'] = model.predict(sample_dmatrix)

# Simpan hasil prediksi ke CSV
df_sample[['id', 'price']].to_csv("prediksi_sample_submission.csv", index=False)
print("Prediksi harga untuk sample_submission telah disimpan dalam 'prediksi_sample_submission.csv'")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

# 1. Load Data
data = pd.read_csv('final_dataset.csv')

# Pastikan ada kolom 'price' dan 'id'
if 'price' not in data.columns or 'id' not in data.columns:
    raise ValueError("Kolom 'price' atau 'id' tidak ditemukan dalam dataset.")

# Parsing tanggal dengan format yang benar
try:
    data['date'] = pd.to_datetime(data['id'].str.split('/').str[2], errors='coerce')
except Exception as e:
    raise ValueError(f"Error dalam parsing tanggal: {e}")

# Hapus baris dengan tanggal NaT (tidak valid)
data = data.dropna(subset=['date'])

# Pastikan ada data setelah preprocessing
if data.empty:
    raise ValueError("Dataset kosong setelah preprocessing!")

# Set indeks ke date dan urutkan
data.set_index('date', inplace=True)
data.sort_index(inplace=True)

# 2. Feature Engineering
def create_features(df):
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['year'] = df.index.year
    df['dayofweek'] = df.index.dayofweek
    df['dayofyear'] = df.index.dayofyear
    df['weekofyear'] = df.index.isocalendar().week
    return df

data = create_features(data)

# Create lag features
for i in range(1, 8):
    data[f'lag_{i}'] = data['price'].shift(i)

# Drop missing values setelah membuat fitur lag
data.dropna(inplace=True)

# Pastikan dataset tidak kosong setelah preprocessing
if data.empty:
    raise ValueError("Dataset kosong setelah pembuatan fitur lag!")

# 3. Split Data
X = data.drop(columns=['price', 'id'])
y = data['price']

# Cek kembali ukuran dataset sebelum split
if X.shape[0] == 0:
    raise ValueError("Dataset kosong sebelum train-test split!")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# 4. Model Training
model = XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# 5. Evaluate Model
y_pred = model.predict(X_test)

# Hitung MSE dan RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'MSE: {mse:.2f}')
print(f'RMSE: {rmse:.2f}')


In [ ]:
# Ambil hanya baris dengan id mengandung "bawang merah"
bawang_merah_data = data[data['id'].str.contains('bawang merah', case=False, na=False)].copy()

# Ambil 10 baris pertama dari dataset yang sudah difilter
bawang_merah_10 = bawang_merah_data.iloc[:10].copy()

# Ambil fitur yang sesuai (hapus kolom 'price' dan 'id')
X_bawang_merah = bawang_merah_10.drop(columns=['price', 'id'])

# Lakukan prediksi
y_pred_bawang_merah = model.predict(X_bawang_merah)

# Buat DataFrame hasil prediksi dengan kolom id yang asli
predictions_df = bawang_merah_10[['id']].copy()
predictions_df['price'] = y_pred_bawang_merah  # Simpan hasil prediksi di kolom price

# Tampilkan hasil prediksi hanya untuk bawang merah
print(predictions_df)


In [ ]:

# --------------------------------------------------------------
# *** Prediksi untuk Data Baru (tahun 2024) ***
# --------------------------------------------------------------

# Load dataset untuk prediksi
df_new = pd.read_csv('sample_submission.csv')

# Pastikan semua fitur yang digunakan dalam training ada di dataset baru
missing_cols = set(X_train.columns) - set(df_new.columns)
if missing_cols:
    print(f"Peringatan: Kolom berikut tidak ditemukan dalam data baru: {missing_cols}")
    for col in missing_cols:
        df_new[col] = 0  # Isi kolom yang hilang dengan nol agar sesuai

# Pastikan dataset baru hanya menggunakan kolom yang ada di X_train
X_new = df_new[X_train.columns]

# Konversi ke DMatrix dan prediksi
new_dmatrix = xgb.DMatrix(data=X_new)
y_pred_new = model.predict(new_dmatrix)

# Masukkan hasil prediksi ke dalam dataset baru
df_new['price'] = y_pred_new
df_new
# Simpan hasil prediksi ke CSV
#df_new[['id', 'price']].to_csv("prediksi2.csv", index=False)

#print("Prediksi harga untuk tahun 2024 telah disimpan dalam 'predicted_prices_2024.csv'")

COBA YUTUB

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# Load dataset
df = pd.read_csv('final_dataset.csv')

# Extract 'jenis', 'provinsi', and 'tanggal' from 'ID'
df[['jenis', 'provinsi', 'tanggal']] = df['id'].str.split('/', expand=True)
df.drop(columns=['id'], inplace=True)

# Convert 'tanggal' to datetime index
df['tanggal'] = pd.to_datetime(df['tanggal'], format='%Y-%m-%d')
df.set_index('tanggal', inplace=True)
print(df)



In [ ]:
df.isna().sum()


In [ ]:
df = df.fillna(df.median(numeric_only=True))
print(df)
df.info()

In [ ]:
df['provinsi_encoded'] = df.groupby('provinsi')['price'].transform('mean')
df['jenis_encoded'] = df.groupby('jenis')['price'].transform('mean')

# Simpan mapping sebelum encoding dihapus
provinsi_map = df[['provinsi', 'provinsi_encoded']].drop_duplicates().set_index('provinsi_encoded')['provinsi'].to_dict()
jenis_map = df[['jenis', 'jenis_encoded']].drop_duplicates().set_index('jenis_encoded')['jenis'].to_dict()

# Hapus kolom aslinya setelah menyimpan mapping
df.drop(columns=['provinsi', 'jenis'], inplace=True)
print(df)

In [ ]:
y = df['price'].copy()
X = df.drop('price', axis=1).copy()
scaler = StandardScaler()

X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=20)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=20)

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {'learning_rate': 0.001, 'max_depth': 6}

model = xgb.train(params, dtrain, evals=[(dval, 'eval')], num_boost_round=10000, early_stopping_rounds=10, verbose_eval=False)
y_true = np.array(y_test)

In [ ]:
y_pred = np.array(model.predict(dtest))
print("R^2 Score: {:.4f}".format(r2_score(y_true, y_pred)))

# Hitung MAE
mae = mean_absolute_error(y_true, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Hitung RMSE
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

In [ ]:

# Kembalikan encoding ke nama asli setelah training
df['provinsi'] = df['provinsi_encoded'].map(provinsi_map)
df['jenis'] = df['jenis_encoded'].map(jenis_map)

# Gabungkan kembali ke ID
df['id'] = df['jenis'] + "/" + df['provinsi'] + "/" + df.index.strftime('%Y-%m-%d')

# Hanya simpan kolom ID dan price
df_final = df[['id', 'price']].reset_index(drop=True)
df_final

In [ ]:
# Load dataset untuk prediksi
df_new = pd.read_csv('sample_submission.csv')

# Pastikan dataset baru hanya memiliki fitur yang digunakan dalam training
X_new = df_new.drop(columns=['id'], errors='ignore')  # Hapus kolom 'id' untuk sementara

# Cek apakah ada kolom yang hilang dari dataset baru
missing_cols = set(X_train.shape[1]) - set(X_new.shape[1])
if missing_cols:
    print(f"Peringatan: Kolom berikut tidak ditemukan dalam data baru: {missing_cols}")
    for col in missing_cols:
        X_new[col] = 0  # Isi kolom yang hilang dengan nol agar sesuai

# Pastikan dataset baru memiliki urutan kolom yang sama dengan X_train
X_new = X_new[X_train.columns]

# Transformasi menggunakan scaler yang sama seperti training
X_new_scaled = scaler.transform(X_new)

# Konversi ke DMatrix dan prediksi
new_dmatrix = xgb.DMatrix(X_new_scaled)
y_pred_new = model.predict(new_dmatrix)

# Simpan hasil prediksi ke dataset sample_submission
df_new['price'] = y_pred_new

# Simpan ke CSV dengan hanya kolom 'id' dan 'price'
df_new[['id', 'price']].to_csv("prediksi2.csv", index=False)

print("Prediksi harga untuk tahun 2024 telah disimpan dalam 'prediksi2.csv'")

In [ ]:
# Load dataset
df = pd.read_csv('final_dataset.csv')

# Ekstrak 'jenis', 'provinsi', dan 'tanggal' dari 'id'
df[['jenis', 'provinsi', 'tanggal']] = df['id'].str.split('/', expand=True)
df.drop(columns=['id'], inplace=True)

# Konversi tanggal menjadi indeks datetime
df['tanggal'] = pd.to_datetime(df['tanggal'], format='%Y-%m-%d')
df.set_index('tanggal', inplace=True)

# Isi nilai NaN dengan median dari setiap kolom numerik
df = df.fillna(df.median(numeric_only=True))

# Encode 'provinsi' dan 'jenis' dengan rata-rata harga
df['provinsi_encoded'] = df.groupby('provinsi')['price'].transform('mean')
df['jenis_encoded'] = df.groupby('jenis')['price'].transform('mean')

# Simpan mapping untuk membalik encoding
provinsi_map = df[['provinsi', 'provinsi_encoded']].drop_duplicates().set_index('provinsi_encoded')['provinsi'].to_dict()
jenis_map = df[['jenis', 'jenis_encoded']].drop_duplicates().set_index('jenis_encoded')['jenis'].to_dict()

# Hapus kolom asli 'provinsi' dan 'jenis'
df.drop(columns=['provinsi', 'jenis'], inplace=True)

In [ ]:

# Pisahkan fitur (X) dan target (y)
y = df['price'].copy()
X = df.drop(columns=['price']).copy()

# Normalisasi fitur menggunakan StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data menjadi training (70%) dan testing (30%)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, train_size=0.7, random_state=20)

# Split training menjadi training (80%) dan validation (20%)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=20)

# Konversi ke DMatrix untuk XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)


# Parameter model
params = {
    'learning_rate': 0.001,  # Lebih tinggi dari sebelumnya untuk konvergensi lebih cepat
    'max_depth': 6,
    'objective': 'reg:squarederror',
    'eval_metric': 'mae'
}

# Training XGBoost model
model = xgb.train(params, dtrain, evals=[(dval, 'eval')], num_boost_round=10000,  early_stopping_rounds=10, verbose_eval=False)

In [ ]:

# Prediksi pada data uji
y_pred = model.predict(dtest)

# Hitung metrik evaluasi
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R^2 Score: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

In [ ]:

# Load dataset untuk prediksi
df_new = pd.read_csv('sample_submission.csv')

# Pastikan dataset baru hanya memiliki fitur yang digunakan dalam training
X_new = df_new.drop(columns=['id'], errors='ignore')  # Hapus ID sementara

# Tangani fitur yang hilang
missing_cols = set(df.drop(columns=['price']).columns) - set(X_new.columns)
if missing_cols:
    print(f"Peringatan: Kolom berikut tidak ditemukan dalam data baru: {missing_cols}")
    for col in missing_cols:
        X_new[col] = 0  # Isi fitur yang hilang dengan nol

# Urutkan kolom agar sesuai dengan X_train
X_new = X_new[X.columns]

# Normalisasi dataset baru dengan scaler yang sama
X_new_scaled = scaler.transform(X_new)

# Prediksi menggunakan model XGBoost
new_dmatrix = xgb.DMatrix(X_new_scaled)
y_pred_new = model.predict(new_dmatrix)

# Simpan hasil prediksi
df_new['price'] = y_pred_new

# Simpan ke CSV dengan hanya kolom 'id' dan 'price'
df_new[['id', 'price']].to_csv("prediksi2.csv", index=False)

print("✅ Prediksi harga untuk tahun 2024 telah disimpan dalam 'prediksi2.csv'")

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("final_dataset.csv")

# Pisahkan 'id' menjadi 'pangan', 'provinsi', dan 'tanggal'
df[['pangan', 'provinsi', 'tanggal']] = df['id'].str.split('/', expand=True)
df['tanggal'] = pd.to_datetime(df['tanggal'], format='%Y-%m-%d')

# Tambahkan fitur tanggal
df['year'] = df['tanggal'].dt.year
df['month'] = df['tanggal'].dt.month
df['day'] = df['tanggal'].dt.day

# Tangani missing values pada 'price' dengan median
df['price'].fillna(df['price'].median(), inplace=True)

# Encoding kategori berdasarkan rata-rata harga
df['provinsi_encoded'] = df.groupby('provinsi')['price'].transform('mean')
df['pangan_encoded'] = df.groupby('pangan')['price'].transform('mean')

# Simpan mapping sebelum encoding dihapus
provinsi_map = df[['provinsi', 'provinsi_encoded']].drop_duplicates().set_index('provinsi_encoded')['provinsi'].to_dict()
pangan_map = df[['pangan', 'pangan_encoded']].drop_duplicates().set_index('pangan_encoded')['pangan'].to_dict()

# Hapus kolom aslinya setelah menyimpan mapping
df.drop(columns=['provinsi', 'pangan', 'tanggal', 'id'], inplace=True)

# Pisahkan fitur (X) dan target (y)
y = df['price'].copy()
X = df.drop(columns=['price']).copy()

# Standarisasi fitur
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split dataset menjadi train, validation, dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=20)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=20)

# Konversi ke DMatrix untuk XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

# Parameter XGBoost
params = {'learning_rate': 0.001, 'max_depth': 6, 'objective': 'reg:squarederror', 'eval_metric': 'rmse'}

# Train model
model = xgb.train(params, dtrain, evals=[(dval, 'eval')], num_boost_round=10000, early_stopping_rounds=10, verbose_eval=False)

# Prediksi pada data uji
y_pred = model.predict(dtest)

# Evaluasi performa
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")


In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("final_dataset.csv")

# Pisahkan 'id' menjadi 'pangan', 'provinsi', dan 'tanggal'
df[['pangan', 'provinsi', 'tanggal']] = df['id'].str.split('/', expand=True)
df['tanggal'] = pd.to_datetime(df['tanggal'], format='%Y-%m-%d')

# Tambahkan fitur tanggal
df['year'] = df['tanggal'].dt.year
df['month'] = df['tanggal'].dt.month
df['day'] = df['tanggal'].dt.day

# Tangani missing values pada 'price' dengan median
df['price'].fillna(df['price'].median(), inplace=True)

# Encoding kategori berdasarkan rata-rata harga
df['provinsi_encoded'] = df.groupby('provinsi')['price'].transform('mean')
df['pangan_encoded'] = df.groupby('pangan')['price'].transform('mean')

# Simpan mapping sebelum encoding dihapus
provinsi_map = df[['provinsi', 'provinsi_encoded']].drop_duplicates().set_index('provinsi_encoded')['provinsi'].to_dict()
pangan_map = df[['pangan', 'pangan_encoded']].drop_duplicates().set_index('pangan_encoded')['pangan'].to_dict()

# Hapus kolom aslinya setelah menyimpan mapping
df.drop(columns=['provinsi', 'pangan', 'tanggal', 'id'], inplace=True)

# Pisahkan fitur (X) dan target (y)
y = df['price'].copy()
X = df.drop(columns=['price']).copy()

# Standarisasi fitur
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split dataset menjadi train, validation, dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=20)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=20)

# Konversi ke DMatrix untuk XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

# Parameter XGBoost
params = {'learning_rate': 0.05, 'max_depth': 7, 'objective': 'reg:squarederror', 'eval_metric': 'rmse'}

# Train model
model = xgb.train(params, dtrain, evals=[(dval, 'eval')], num_boost_round=200, early_stopping_rounds=10, verbose_eval=False)

# Prediksi pada data uji
y_pred = model.predict(dtest)

# Evaluasi performa
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

# Cek distribusi hasil prediksi
sns.histplot(y_pred, bins=30)
plt.title("Distribusi Prediksi Harga")
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("final_dataset.csv")

# Pisahkan 'id' menjadi 'pangan', 'provinsi', dan 'tanggal'
df[['pangan', 'provinsi', 'tanggal']] = df['id'].str.split('/', expand=True)
df['tanggal'] = pd.to_datetime(df['tanggal'], format='%Y-%m-%d')

# Tambahkan fitur tanggal
df['year'] = df['tanggal'].dt.year
df['month'] = df['tanggal'].dt.month
df['day'] = df['tanggal'].dt.day

# Tangani missing values pada 'price' dengan median
df['price'].fillna(df['price'].median(), inplace=True)

# Encoding kategori berdasarkan rata-rata harga
df['provinsi_encoded'] = df.groupby('provinsi')['price'].transform('mean')
df['pangan_encoded'] = df.groupby('pangan')['price'].transform('mean')

# Simpan mapping sebelum encoding dihapus
provinsi_map = df[['provinsi', 'provinsi_encoded']].drop_duplicates().set_index('provinsi_encoded')['provinsi'].to_dict()
pangan_map = df[['pangan', 'pangan_encoded']].drop_duplicates().set_index('pangan_encoded')['pangan'].to_dict()

# Hapus kolom aslinya setelah menyimpan mapping
df.drop(columns=['provinsi', 'pangan', 'tanggal', 'id'], inplace=True)

# Pisahkan fitur (X) dan target (y)
y = df['price'].copy()
X = df.drop(columns=['price']).copy()

# Simpan nama fitur sebelum transformasi
feature_names = X.columns

# Standarisasi fitur
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split dataset menjadi train, validation, dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=20)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=20)

# Konversi ke DMatrix untuk XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

# Parameter XGBoost
params = {'learning_rate': 0.001, 'max_depth': 6, 'objective': 'reg:squarederror', 'eval_metric': 'rmse'}

# Train model
model = xgb.train(params, dtrain, evals=[(dval, 'eval')], num_boost_round=10000, early_stopping_rounds=10, verbose_eval=False)

# Prediksi pada data uji
y_pred = model.predict(dtest)

# Evaluasi performa
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

# Membaca sample submission dan mengisi prediksi
sample_submission = pd.read_csv("sample_submission.csv")
X_submission = df[df.columns.difference(['price'])].copy()  # Gunakan fitur yang sama

# Pastikan urutan fitur sama seperti saat fit_transform
X_submission = X_submission[feature_names]
X_submission = scaler.transform(X_submission)  # Standarisasi

dsubmission = xgb.DMatrix(X_submission)
sample_submission['price'] = model.predict(dsubmission)

# Simpan hasil prediksi
sample_submission.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv")


In [ ]:
!pip install imbalanced-learn

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [ ]:
# Load dataset
df = pd.read_csv("final_dataset.csv")

# Pisahkan 'id' menjadi 'pangan', 'provinsi', 'tanggal'
df[['pangan', 'provinsi', 'tanggal']] = df['id'].str.split('/', expand=True)
df['tanggal'] = pd.to_datetime(df['tanggal'], format='%Y-%m-%d')

# Tambahkan fitur tanggal
df['year'] = df['tanggal'].dt.year
df['month'] = df['tanggal'].dt.month
df['day'] = df['tanggal'].dt.day
df['year_month'] = df['year'] * 100 + df['month']  # Fitur kombinasi tahun-bulan

# Tangani missing values dengan median
df.fillna({'price': df['price'].median()}, inplace=True)

# Tambahkan fitur harga sebelumnya berdasarkan 'pangan' dan 'provinsi'
df['prev_price'] = df.groupby(['pangan', 'provinsi'])['price'].shift(1)
df['prev_price'].fillna(df['price'].median(), inplace=True)

# Encoding kategori
df = pd.get_dummies(df, columns=['pangan', 'provinsi'])

# Transformasi log harga untuk mengurangi skala ekstrem
df['price'] = np.log1p(df['price'])

# Pisahkan fitur (X) dan target (y)
X = df.drop(columns=['id', 'tanggal', 'price'])
y = df['price']


In [ ]:
# Split dataset menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# SMOTE untuk menangani data imbalance (opsional)
smote = SMOTE(random_state=123)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Konversi ke DMatrix untuk XGBoost
train_dmatrix = xgb.DMatrix(data=X_train, label=y_train)
test_dmatrix = xgb.DMatrix(data=X_test)

# Hyperparameter tuning
params = {
    'objective': 'reg:squarederror',
    'max_depth': 7,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'n_estimators': 300,
    'gamma': 0.1,
    'lambda': 1,
    'eval_metric': 'rmse'
}

num_round = 200
model = xgb.train(
    params, 
    train_dmatrix, 
    num_round, 
    evals=[(train_dmatrix, "train"), (test_dmatrix, "test")],
    early_stopping_rounds=10,  # Early stopping untuk menghindari overfitting
    verbose_eval=10
)

# Train model dengan cross-validation
cv_results = xgb.cv(params, train_dmatrix, num_boost_round=200, nfold=5, metrics="rmse", as_pandas=True, seed=123)
print(cv_results.tail())

In [ ]:

# Train final model
num_round = 200
model = xgb.train(params, train_dmatrix, num_round)

# Prediksi pada data uji
y_pred_test = model.predict(test_dmatrix)

# Hitung RMSE dan MAE
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae = mean_absolute_error(y_test, y_pred_test)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

In [ ]:

# Prediksi untuk sample_submission.csv
df_new = pd.read_csv('sample_submission.csv')

# Pastikan semua fitur yang digunakan dalam training ada di dataset baru
missing_cols = set(X_train.columns) - set(df_new.columns)
if missing_cols:
    print(f"Peringatan: Kolom berikut tidak ditemukan dalam data baru: {missing_cols}")
    for col in missing_cols:
        df_new[col] = 0  # Isi kolom yang hilang dengan nol agar sesuai

# Pastikan dataset baru hanya menggunakan kolom yang ada di X_train
X_new = df_new[X_train.columns]

# Konversi ke DMatrix dan prediksi
new_dmatrix = xgb.DMatrix(data=X_new)
y_pred_new = model.predict(new_dmatrix)

# Kembalikan hasil prediksi dari log transformasi
df_new['price'] = np.expm1(y_pred_new)

# Simpan hasil prediksi ke CSV
df_new[['id', 'price']].to_csv("prediksi.csv", index=False)

print(df_new[['id', 'price']].head())

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error

# 1️⃣ Baca dataset
df = pd.read_csv("final_dataset.csv")

# 2️⃣ Pisahkan ID dan ekstraksi tanggal
df[['pangan', 'provinsi', 'tanggal']] = df['id'].str.split('/', expand=True)
df['tanggal'] = pd.to_datetime(df['tanggal'], format='%Y-%m-%d')

# 3️⃣ Tambahkan fitur tanggal: year, month, day
df['year'] = df['tanggal'].dt.year
df['month'] = df['tanggal'].dt.month
df['day'] = df['tanggal'].dt.day

# 4️⃣ Tangani missing values pada kolom 'price' dengan median
df.fillna({'price': df['price'].median()}, inplace=True)

# 5️⃣ Encoding kategori menggunakan one-hot encoding
df = pd.get_dummies(df, columns=['pangan', 'provinsi'])

# 6️⃣ Pisahkan fitur (X) dan target (y)
X = df.drop(columns=['id', 'tanggal', 'price'])
y = df['price']

# 7️⃣ Fungsi untuk menghitung MAPE
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# 8️⃣ K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=123)
params = {
    'objective': 'reg:squarederror',
    'max_depth': 10,   # Coba nilai lebih besar
    'learning_rate': 0.1,  # Coba tingkatkan
    'eval_metric': 'rmse'
}

mape_scores = []

for train_idx, val_idx in kf.split(X):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    train_dmatrix = xgb.DMatrix(X_train, label=y_train)
    val_dmatrix = xgb.DMatrix(X_val, label=y_val)

    # Train model dengan early stopping
    model = xgb.train(
        params, 
        train_dmatrix, 
        num_boost_round=200, 
        evals=[(train_dmatrix, "train"), (val_dmatrix, "val")], 
        early_stopping_rounds=10,
        verbose_eval=10
    )

    # Predict validation set
    y_pred_val = model.predict(val_dmatrix)

    # Calculate MAPE
    mape_score = mape(y_val, y_pred_val)
    mape_scores.append(mape_score)

# 9️⃣ Hasil Cross Validation
print(f"Avg MAPE: {np.mean(mape_scores):.2f}% ± {np.std(mape_scores):.2f}%")

# 🔟 Train Model Final dengan seluruh data
train_dmatrix = xgb.DMatrix(X, label=y)
final_model = xgb.train(params, train_dmatrix, num_boost_round=200)

# 1️⃣1️⃣ Prediksi untuk sample_submission.csv
df_new = pd.read_csv('sample_submission.csv')

# Pastikan semua fitur yang digunakan dalam training ada di dataset baru
missing_cols = set(X.columns) - set(df_new.columns)
if missing_cols:
    print(f"Peringatan: Kolom berikut tidak ditemukan dalam data baru: {missing_cols}")
    for col in missing_cols:
        df_new[col] = 0  # Isi kolom yang hilang dengan nol agar sesuai

# Pastikan dataset baru hanya menggunakan kolom yang ada di X
X_new = df_new[X.columns]

# Konversi ke DMatrix dan prediksi
new_dmatrix = xgb.DMatrix(data=X_new)
y_pred_new = final_model.predict(new_dmatrix)

# Masukkan hasil prediksi ke dalam dataset baru
df_new['price'] = y_pred_new

# Simpan hasil prediksi ke CSV
df_new[['id', 'price']].to_csv("prediksi3.csv", index=False)


print("Prediksi berhasil disimpan ke prediksi.csv!")


[0]	train-rmse:32810.64179	val-rmse:32763.11413
[10]	train-rmse:32500.63035	val-rmse:32453.93162
[20]	train-rmse:32193.84739	val-rmse:32147.87935
[30]	train-rmse:31890.25606	val-rmse:31845.00673
[40]	train-rmse:31589.82857	val-rmse:31545.26243
[50]	train-rmse:31292.54068	val-rmse:31248.63877
[60]	train-rmse:30998.35766	val-rmse:30955.12616
[70]	train-rmse:30707.25982	val-rmse:30664.67073
[80]	train-rmse:30419.21114	val-rmse:30377.23076
[90]	train-rmse:30134.17382	val-rmse:30092.78322
[100]	train-rmse:29852.11276	val-rmse:29811.34544
[110]	train-rmse:29573.00705	val-rmse:29532.90211
[120]	train-rmse:29296.85896	val-rmse:29257.42416
[130]	train-rmse:29023.61436	val-rmse:28984.84410
[140]	train-rmse:28753.19255	val-rmse:28715.03237
[150]	train-rmse:28485.61986	val-rmse:28448.08145
[160]	train-rmse:28220.85697	val-rmse:28183.93084
[170]	train-rmse:27958.87705	val-rmse:27922.52877
[180]	train-rmse:27699.65102	val-rmse:27663.91495
[190]	train-rmse:27443.17003	val-rmse:27408.04645
[199]	train

In [ ]:
df_new